# Pipeline of Digits

This is a starting notebook for solving the "Pipeline of Digits" assignment.


This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

Let's make sure we are running the latest version of the SakeMaker's SDK. **Restart the notebook** after you upgrade the library.

In [2]:
# !pip install -q --upgrade awscli
# !pip install -q --upgrade pip
# !pip install -q --upgrade sagemaker
# !pip show sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.145.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 6.3.0 which is incompatible.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.118 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is

In [3]:
%load_ext autoreload
%autoreload 2

In [84]:
import boto3
import sagemaker
import pandas as pd

from pathlib import Path

iam_client = boto3.client("iam")
sagemaker_client = boto3.client("sagemaker")
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

## Creating the S3 Bucket

Let's create an S3 bucket where you will upload all the information generated by the pipeline. Make sure you set `BUCKET` to the name of the bucket you want to use. This name has to be unique.

If you want to create a bucket in a region other than `us-east-1`, use this command instead:

```
!aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint=$region
```

The `LocationConstraint` argument should specify the region where you want to create the bucket.

In [5]:
# uncomment if you want create new bucket

BUCKET = "mlschool-mnist"
region = "eu-north-1"

In [6]:
# !aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint=$region

In [7]:
import os
current_folder = os.getcwd()
print(current_folder)
%ls

/root/ml-school/mnist
dataset/        evaluation.py     mnist.ipynb      train.py
dataset.tar.gz  lambda_helper.py  preprocessor.py


## Loading the dataset

We have two CSV files containing the MNIST dataset. These files come from the [MNIST in CSV](https://www.kaggle.com/datasets/oddrationale/mnist-in-csv) Kaggle dataset.

The `mnist_train.csv` file contains 60,000 training examples and labels. The `mnist_test.csv` contains 10,000 test examples and labels. Each row consists of 785 values: the first value is the label (a number from 0 to 9) and the remaining 784 values are the pixel values (a number from 0 to 255).

Let's extract the `dataset.tar.gz` file.

In [8]:
MNIST_FOLDER = "/root/ml-school/mnist"
DATASET_FOLDER = Path(MNIST_FOLDER) / "dataset"

!tar -xvzf $MNIST_FOLDER/dataset.tar.gz -C $MNIST_FOLDER --no-same-owner

dataset/
dataset/mnist_test.csv
dataset/mnist_train.csv


Let's load the first 10 rows of the test set.

In [9]:
df = pd.read_csv(DATASET_FOLDER / "mnist_train.csv", nrows=10)
df

label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0      5    0    0    0    0    0    0    0    0    0  ...      0      0   
1      0    0    0    0    0    0    0    0    0    0  ...      0      0   
2      4    0    0    0    0    0    0    0    0    0  ...      0      0   
3      1    0    0    0    0    0    0    0    0    0  ...      0      0   
4      9    0    0    0    0    0    0    0    0    0  ...      0      0   
5      2    0    0    0    0    0    0    0    0    0  ...      0      0   
6      1    0    0    0    0    0    0    0    0    0  ...      0      0   
7      3    0    0    0    0    0    0    0    0    0  ...      0      0   
8      1    0    0    0    0    0    0    0    0    0  ...      0      0   
9      4    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0  
5      0      0      0      0      0      0      0      0  
6      0      0      0      0      0      0      0      0  
7      0      0      0      0      0      0      0      0  
8      0      0      0      0      0      0      0      0  
9      0      0      0      0      0      0      0      0  

[10 rows x 785 columns]

## Uploading dataset to S3

In [10]:
S3_FILEPATH = f"s3://mlschool-mnist/{MNIST_FOLDER}"


TRAIN_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / "mnist_train.csv"), 
    desired_s3_uri=S3_FILEPATH,
)

TEST_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / "mnist_test.csv"), 
    desired_s3_uri=S3_FILEPATH,
)

print(f"Train set S3 location: {TRAIN_SET_S3_URI}")
print(f"Test set S3 location: {TEST_SET_S3_URI}")

Train set S3 location: s3://mlschool-mnist/root/ml-school/mnist/mnist_train.csv
Test set S3 location: s3://mlschool-mnist/root/ml-school/mnist/mnist_test.csv


In [11]:
%%writefile {MNIST_FOLDER}/preprocessor.py

import os
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pickle import dump
from pathlib import Path


# This is the location where the SageMaker Processing job
# will save the input dataset.
BASE_DIR = "/opt/ml/processing"
DATA_FILEPATH_TRAIN = Path(BASE_DIR) / "input" / "mnist_train" / "mnist_train.csv"
DATA_FILEPATH_TEST = Path(BASE_DIR) / "input" / "mnist_test" / "mnist_test.csv"


def save_splits(base_dir, train, validation, test):
    """
    One of the goals of this script is to output the three
    dataset splits. This function will save each of these
    splits to disk.
    """

    train_path = Path(base_dir) / "train"
    validation_path = Path(base_dir) / "validation"
    test_path = Path(base_dir) / "test"

    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)


def save_pipeline(base_dir, pipeline):
    """
    Saves the Scikit-Learn pipeline that we used to
    preprocess the data.
    """
    pipeline_path = Path(base_dir) / "pipeline"
    pipeline_path.mkdir(parents=True, exist_ok=True)
    dump(pipeline, open(pipeline_path / "pipeline.pkl", 'wb'))


def generate_baseline(base_dir, X_train, y_train):
    """
    Generates a baseline for our model using the train set.
    It saves the baseline in a JSON file where every line is
    a JSON object.
    """
    baseline_path = Path(base_dir) / "baseline"
    baseline_path.mkdir(parents=True, exist_ok=True)

    df = X_train.copy()
    df["groundtruth"] = y_train

    df.to_json(baseline_path / "baseline.json", orient='records', lines=True)


def preprocess(base_dir, data_filepath_train, data_filepath_test):
    """
    Preprocesses the supplied raw dataset and splits it into a train, validation,
    and a test set.
    """

    df_train = pd.read_csv(data_filepath_train, nrows=7200)
    df_test = pd.read_csv(data_filepath_test, nrows=2000)

    numerical_columns = df_train.select_dtypes(include=['number']).drop(['label'], axis=1).columns

    numerical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical", numerical_preprocessor, numerical_columns),
        ]
    )

    X_train = df_train.copy()
    y_train = df_train['label']
    columns = list(X_train.drop(['label'], axis=1).columns)

    X_train, X_validation, y_train, y_validation =  train_test_split(X_train, y_train, test_size=0.2, random_state=12)
    X_test = df_test.copy()

    y_train = X_train.label
    y_validation = X_validation.label
    y_test = X_test.label

    X_train.drop(["label"], axis=1, inplace=True)
    X_validation.drop(["label"], axis=1, inplace=True)
    X_test.drop(["label"], axis=1, inplace=True)

    X_train = pd.DataFrame(X_train, columns=columns)
    X_validation = pd.DataFrame(X_validation, columns=columns)
    X_test = pd.DataFrame(X_test, columns=columns)

    y_train = y_train.astype(int)
    y_validation = y_validation.astype(int)
    y_test = y_test.astype(int)

    # Let's use the train set to generate a baseline that we can
    # later use to measure the quality of our model. This baseline
    # will use the original data.
    generate_baseline(base_dir, X_train, y_train)

    # Transform the data using the Scikit-Learn pipeline.
    X_train = preprocessor.fit_transform(X_train)
    X_validation = preprocessor.transform(X_validation)
    X_test = preprocessor.transform(X_test)

    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)

    save_splits(base_dir, train, validation, test)
    save_pipeline(base_dir, pipeline=preprocessor)


if __name__ == "__main__":
    preprocess(BASE_DIR, DATA_FILEPATH_TRAIN, DATA_FILEPATH_TEST)

Overwriting /root/ml-school/mnist/preprocessor.py


In [12]:
from botocore.exceptions import ClientError
from sagemaker.inputs import FileSystemInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig

In [13]:
dataset_location_train = ParameterString(
    name="dataset_location_train",
    default_value=TRAIN_SET_S3_URI,
)

dataset_location_test = ParameterString(
    name="dataset_location_test",
    default_value=TEST_SET_S3_URI,
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f"{S3_FILEPATH}/preprocessing",
)

baseline_destination = ParameterString(
    name="baseline_destination",
    default_value=f"{S3_FILEPATH}/baseline",
)

In [14]:
cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

In [15]:
sklearn_processor = SKLearnProcessor(
    base_job_name="mnist-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role
)

In [16]:
preprocess_step = ProcessingStep(
    name="preprocessing",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location_train, destination="/opt/ml/processing/input/mnist_train"),
        ProcessingInput(source=dataset_location_test, destination="/opt/ml/processing/input/mnist_test"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=preprocessor_destination),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=preprocessor_destination),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=preprocessor_destination),
        ProcessingOutput(output_name="pipeline", source="/opt/ml/processing/pipeline", destination=preprocessor_destination),
        ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline", destination=baseline_destination),
    ],
    code=f"{MNIST_FOLDER}/preprocessor.py",
    cache_config=cache_config
    
)

In [17]:
# session1_pipeline = Pipeline(
#     name="mnist-session1-pipeline",
#     parameters=[
#         dataset_location_train,
#         dataset_location_test,
#         preprocessor_destination,
#         baseline_destination,
#     ],
#     steps=[
#         preprocess_step, 
#     ]
# )

In [18]:
# session1_pipeline.upsert(role_arn=role)
# execution = session1_pipeline.start()

In [19]:
# session1_pipeline.delete()

In [20]:
%%writefile {MNIST_FOLDER}/train.py

import os
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from pathlib import Path

# PyTorch model
class MyModel(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(10, hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32, learning_rate=0.01):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)

    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)

    train_dataset = TensorDataset(torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    validation_dataset = TensorDataset(torch.tensor(X_validation.values, dtype=torch.float32), torch.tensor(y_validation.values, dtype=torch.long))
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    model = MyModel(input_size=X_train.shape[1], hidden_size=128, output_size=10)

    # Define the loss function
    loss_fn = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        train_loss = 0.0
        train_acc = 0.0

        model.train()
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()

            y_pred = model(x_batch)

            loss = loss_fn(y_pred, y_batch)
            loss.backward()

            optimizer.step()

            train_loss += loss.item() * x_batch.shape[0]
            train_acc += accuracy_score(y_batch.numpy(), np.argmax(y_pred.detach().numpy(), axis=1)) * x_batch.shape[0]

        train_loss /= len(train_dataset)
        train_acc /= len(train_dataset)

        validation_loss = 0.0
        validation_acc = 0.0

        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in validation_loader:
                y_pred = model(x_batch)

                loss = loss_fn(y_pred, y_batch)

                validation_loss += loss.item() * x_batch.shape[0]
                validation_acc += accuracy_score(y_batch.numpy(), np.argmax(y_pred.numpy(), axis=1)) * x_batch.shape[0]

            validation_loss /= len(validation_dataset)
            validation_acc /= len(validation_dataset)

        print(f"Epoch {epoch+1}: Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}, Validation loss: {validation_loss:.4f}, Validation accuracy: {validation_acc:.4f}")

    model_filepath = Path(base_directory) / "model" / "001"
    torch.save(model.state_dict(), model_filepath)

if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to the entry point
    # as script arguments. SageMaker will also provide a list of special parameters
    # that you can capture here. Here is the full list:
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    parser.add_argument("--learning_rate", type=float, default=0.1)
    args, _ = parser.parse_known_args()

    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size,
        learning_rate=args.learning_rate
    )

Overwriting /root/ml-school/mnist/train.py


In [21]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter
from sagemaker.parameter import ContinuousParameter
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.pytorch import PyTorch
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession

In [79]:
hyperparameters = {
    "epochs": 50,
    "batch_size": 32,
    "learning_rate": 0.1
}

estimator = TensorFlow(
    entry_point=f"{MNIST_FOLDER}/train.py",
    hyperparameters=hyperparameters,
    framework_version="2.4",
    py_version="py37",
    instance_type="ml.m5.large",
    instance_count=1,
    script_mode=True,
    disable_profiler=True,
    role=role,
)

pytorch_estimator = PyTorch(
    entry_point=f"{MNIST_FOLDER}/train.py",
    instance_type='ml.m5.large',
    instance_count=1,
    framework_version='1.8.0',
    py_version='py36',
    hyperparameters = hyperparameters,
    role=role,
    disable_profiler=True,
    script_mode=True,
)

In [80]:
cache_config_train = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

training_step = TrainingStep(
    name="training",
    estimator=pytorch_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config_train
)

In [81]:
hyperparameter_ranges = {
    "epochs": IntegerParameter(30, 35),
    "learning_rate": ContinuousParameter(0.1, 0.2)
}

objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "Validation accuracy: ([0-9\\.]+)"}]
    
tuner = HyperparameterTuner(
    pytorch_estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=4,
    max_parallel_jobs=1
)

In [25]:
cache_config_tunning = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

tuning_step = TuningStep(
    name = "tuning",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config_tunning
)

In [26]:
USE_TUNING_STEP = True

# session2_pipeline = Pipeline(
#     name="mnist-session2-pipeline",
#     parameters=[
#         dataset_location_train,
#         dataset_location_test,
#         preprocessor_destination,
#         baseline_destination,
#     ],
#     steps=[
#         preprocess_step, 
#         tuning_step if USE_TUNING_STEP else training_step
#     ]
# )

In [27]:
# session2_pipeline.upsert(role_arn=role)
# execution = session2_pipeline.start()
# session2_pipeline.delete()

In [28]:
%%writefile {MNIST_FOLDER}/evaluation.py

import os
import argparse

import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import json
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import tarfile

from pathlib import Path

import torch

# PyTorch model
class MyModel(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(10, hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"
# OUTPUT_PATH = "/opt/ml/processing/evaluation/"


def evaluate(model_path, test_path, output_path):
     # The first step is to extract the model package provided
    # by SageMaker.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)

    test_dataset = TensorDataset(torch.tensor(X_test.values, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    # We can now load the model from disk.
    model = MyModel(input_size=X_test.shape[1], hidden_size=128, output_size=10)
    model.load_state_dict(torch.load(Path(model_path) / "001"))

    # Define the loss function
    loss_fn = nn.CrossEntropyLoss()

    test_loss = 0.0
    test_acc = 0.0

    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            y_pred = model(x_batch)

            loss = loss_fn(y_pred, y_batch)

            test_loss += loss.item() * x_batch.shape[0]
            test_acc += accuracy_score(y_batch.numpy(), np.argmax(y_pred.numpy(), axis=1)) * x_batch.shape[0]

        test_loss /= len(test_dataset)
        test_acc /= len(test_dataset)

    print(f"Test accuracy: {test_acc}")

    # Let's add the accuracy of the model to our evaluation report.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": test_acc
            },
            "loss": {
                "value": test_loss
            },
            "n_rows": {
                "value": X_test.shape[1]
            },
        },
    }

    # We need to save the evaluation report to the output path.
    Path(output_path).mkdir(parents=True, exist_ok=True)
    print(f"path output: {Path(output_path)}")
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--output_path", type=str, default="/opt/ml/processing/evaluation/")
    args, _ = parser.parse_known_args()
    print(f"arg outpath {args.output_path}")
    
    evaluate(
        model_path=MODEL_PATH,
        test_path=TEST_PATH,
        output_path=args.output_path
    )

Overwriting /root/ml-school/mnist/evaluation.py


In [29]:
evaluation_destination = ParameterString(
    name="evaluation_destination_1",
    default_value=f'{S3_FILEPATH}/evaluation',
)

In [87]:
# Let's retrieve the image we want to use to run the
# processing job.
image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.11.0",
    py_version="py38",
    image_scope="training",
    instance_type="ml.m5.large"
)

# We can now setup the processor using the URI of
# the pre-built docker image.
evaluation_script_processor = ScriptProcessor(
    base_job_name="mnist-evaluation-processor",
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

In [88]:
# This is the input in case we want to use the best model generated
# by the Tuning Step.
tuning_model_input = ProcessingInput(
    source=tuning_step.get_top_model_s3_uri(
        top_k=1, 
        s3_bucket=sagemaker_session.default_bucket()
    ),
    destination="/opt/ml/processing/model",
)

# This is the input in case we want to use the trained model
# from the Training Step.
training_model_input = ProcessingInput(
    source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    destination="/opt/ml/processing/model"
)

# We can now select the appropriate input depending on which step
# we are using.
model_input = tuning_model_input if USE_TUNING_STEP else training_model_input

In [89]:
from sagemaker.workflow.properties import PropertyFile

cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
evaluation_report = PropertyFile(
    name="evaluation-report",
    output_name="evaluation",
    path="evaluation.json"
)


# Notice how this step uses the model generated by the tuning or training
# step, and the test set generated by the preprocessing step.
evaluation_step = ProcessingStep(
    name="evaluation_pytorch",
    processor=evaluation_script_processor,
    inputs=[
        model_input,
        ProcessingInput(
            source=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation",
                         source="/opt/ml/processing/evaluation",
                         destination=evaluation_destination),
    ],
    code=f"{MNIST_FOLDER}/evaluation.py",
    property_files=[evaluation_report],
    cache_config=cache_config,
    job_arguments=["--output_path", "/opt/ml/processing/evaluation/"]
)



In [90]:
# session3_pipeline = Pipeline(
#     name="mnist-session3-pipeline",
#     parameters=[
#         dataset_location_train,
#         dataset_location_test,
#         preprocessor_destination,
#         baseline_destination,
#         evaluation_destination
#     ],
#     steps=[
#         preprocess_step, 
#         tuning_step if USE_TUNING_STEP else training_step,
#         evaluation_step
#     ]
# )

In [91]:
# session3_pipeline.upsert(role_arn=role)
# execution = session3_pipeline.start()

In [92]:
from sagemaker import ModelPackage
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.predictor import Predictor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join

In [93]:
model_approval_status = ParameterString(
    name="model_approval_status", 
    default_value="Approved"
)

accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.85
)


model_pending_approval_status = ParameterString(
    name="model_approval_status", 
    default_value="PendingManualApproval"
)

accuracy_threshold_pa = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.50
)

In [94]:
# This is the model data in case we want to use the best model generated
# by the Tuning Step.
tuning_model_data = tuning_step.get_top_model_s3_uri(
    top_k=0, 
    s3_bucket=sagemaker_session.default_bucket()
)

# This is the model data in case we want to use the trained model
# from the Training Step.
training_model_data = training_step.properties.ModelArtifacts.S3ModelArtifacts

# We can now select the appropriate model data depending on which step
# we are using.
model_data = tuning_model_data if USE_TUNING_STEP else training_model_data

In [95]:
model = Model(
    image_uri=image_uri,
    model_data=model_data,
    sagemaker_session=PipelineSession(),
    role=role,
)

In [96]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="", values=[
            evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'],
            "/evaluation.json"]
        ),
        content_type="application/json",
    )
)

In [97]:
model_package_group_name = "mnist-model-package-group"

register_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.11.0",
        # sample_payload_url="",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_approval_status,
    ),
)

pa_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="PYTORCH",
        framework_version="1.11.0",
        # sample_payload_url="",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_pending_approval_status,
    ),
)

In [98]:
from sagemaker.workflow.lambda_step import LambdaStep
from sagemaker.lambda_helper import Lambda

step_lambda = LambdaStep(
    name="ProcessingLambda",
    lambda_func=Lambda(
        function_arn="arn:aws:lambda:eu-north-1:284415450706:function:my-example-function"
    )
)

In [99]:
# We can get the model accuracy directly from the evaluation
# report property file.
condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

condition_gte_pa = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold_pa
)

# If the condition succeeds, we can call the Model Step.
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[register_model_step],
    else_steps=[step_lambda],
)

# If the condition succeeds, we can call the Model Step.
condition_step = ConditionStep(
    name="check-model-accuracy_pa",
    conditions=[condition_gte_pa],
    if_steps=[pa_model_step],
    else_steps=[step_lambda],
)

In [100]:
session4_pipeline = Pipeline(
    name="mnist-session4-pipeline",
    parameters=[
        dataset_location_train,
        dataset_location_test, 
        preprocessor_destination,
        baseline_destination,
        evaluation_destination,
        model_approval_status,
        accuracy_threshold,
    ],
    steps=[
        preprocess_step, 
        tuning_step if USE_TUNING_STEP else training_step, 
        evaluation_step,
        condition_step
    ],
)

In [101]:
session4_pipeline.upsert(role_arn=role)
execution = session4_pipeline.start()

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


In [102]:
def get_latest_approved_model_package(model_package_group_name):
    """
    Returns the latest approved model package registered under the 
    specified model package group.
    """
    try:
        # We can use the boto3 SageMaker's API to list the existing
        # model packages with the specified name. We only care about
        # approved models.
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=20,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # If we get a NextToken back, we need to deal with pagination.
        while len(approved_packages) == 0 and "NextToken" in response:
            response = sagemaker_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=20,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        if len(approved_packages) == 0:
            print(f"No approved model pacakages for \"{model_package_group_name}\"")
            return None

        # At this point we identified the latest approved model,
        # so we can return it.
        print(f"Latest approved model package: {approved_packages[0]['ModelPackageArn']}")
        return approved_packages[0]

    except ClientError as e:
        print(e.response["Error"]["Message"])
        raise Exception(e.response["Error"]["Message"])

In [103]:
approved_model_package = get_latest_approved_model_package(model_package_group_name)
model_description = None

if approved_model_package:
    approved_model_package_arn = approved_model_package["ModelPackageArn"]

    model_description = sagemaker_client.describe_model_package(
        ModelPackageName=approved_model_package_arn
    )

model_description

Latest approved model package: arn:aws:sagemaker:eu-north-1:284415450706:model-package/mnist-model-package-group/1


{'ModelPackageGroupName': 'mnist-model-package-group',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:eu-north-1:284415450706:model-package/mnist-model-package-group/1',
 'CreationTime': datetime.datetime(2023, 5, 2, 16, 53, 3, 447000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.eu-north-1.amazonaws.com/pytorch-training:1.12.0-cpu-py38',
    'ImageDigest': 'sha256:baac8a6f928138efb00f54c11f5cee0e86090a586656d2b502881e80224bc6a7',
    'ModelDataUrl': 's3://sagemaker-eu-north-1-284415450706/1u35u9mbvmrh-tuning-p70F2izIVi-003-02acea4f/output/model.tar.gz',
    'Environment': {},
    'Framework': 'PYTORCH',
    'FrameworkVersion': '1.12.0'}],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.m5.large'],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForM

In [106]:
model_package = ModelPackage(
    model_package_arn=approved_model_package_arn, 
    sagemaker_session=sagemaker_session,
    role=role, 
)

endpoint_name = "mnist-endpoint"

model_package.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1, 
    instance_type="ml.m5.large", 
)

------

KeyboardInterrupt: 

In [ ]:
predictor = Predictor(endpoint_name=endpoint_name)

# The payload we need to provide the model is in CSV format. Notice how the model expects data that's
# already transformed. We can't provide the original data from our dataset because the model will not
# work with it.
payload = "2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,170,254,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,191,245,254,254,175,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,70,242,254,235,233,254,182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,184,254,243,99,27,110,254,182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,205,247,98,26,0,0,110,254,149,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,104,20,0,0,0,0,110,254,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110,234,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,214,149,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,134,200,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,231,144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,146,244,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,201,132,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,164,226,31,0,0,0,0,0,0,0,11,122,205,47,0,0,0,0,0,0,0,0,0,0,0,0,0,79,255,128,0,0,0,0,0,0,15,84,216,250,144,12,0,0,0,0,0,0,0,0,0,0,0,0,7,207,228,37,0,0,0,0,20,136,238,254,228,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,135,254,66,0,0,0,53,190,254,254,197,117,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,231,182,0,32,101,184,249,239,179,96,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128,254,99,128,232,254,251,185,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,221,254,254,254,254,192,70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,254,254,188,105,72,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0"
response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})

# We can decode the output of the endpoint and print the "predictions" key.
predictions = json.loads(response.decode("utf-8"))["predictions"]
print(f"Prediction: {np.argmax(predictions, axis=1)[0]}")

In [53]:
# session2_pipeline.upsert(role_arn=role)
session2_pipeline.delete()